In [1]:
def create_df_wikipedia():
    file = open("wikiElec.ElecBs3.txt","r",encoding="ANSI")
    lines = file.readlines()
    ## Split file on \n (to seperate elections)
    split_idxs = [idx for idx, val in enumerate(lines) if val == '\n']
    election_lines = []
    from_index = 0
    for index in split_idxs:
        election_lines.append(lines[from_index:index+1])
        from_index=index
    ## OMMIT THE RULE FROM THE FIRST ELECTION_LINES 
    election_lines[0] = election_lines[0][6:]

    ## create the wikipedia_df by appending all election dataframes
    wikipedia_df = pd.DataFrame(columns=["from_node","to_node","sign"])
    for i in range(len(election_lines)):
        new_df = create_df_initiated_vote(election_lines[i])
        wikipedia_df=wikipedia_df.append(new_df, ignore_index=True)
    wikipedia_df = wikipedia_df[wikipedia_df.sign != 0]
    wikipedia_df.sort_values(by=["from_node", "to_node"]).reset_index(drop=True)
    return wikipedia_df

## function for wikipedia dataset to parse an election and create a seperate df for each one
def create_df_initiated_vote(election_lines):
    df_list = []
    for line in election_lines:
        if (line[0]=='U'):
            ints = [int(s) for s in line.split() if s.lstrip('-').isdigit()]
            to_node = ints[0]
        if (line[0]=='V'):
            ints = [int(s) for s in line.split() if s.lstrip('-').isdigit()]
            from_node= ints[1]
            sign = ints[0]
            df_list.append([from_node,to_node,sign])
    df = pd.DataFrame(df_list,columns=["from_node","to_node","sign"])
    return df

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

In [3]:
def load_data_create_graph(path):
    if path!="wikiElec.ElecBs3.txt":
        dataset = np.loadtxt(path)
        dataset=dataset.astype("int64")
        df = pd.DataFrame(dataset,columns=['from_node','to_node','sign'])
        graph = nx.from_pandas_edgelist(df, "from_node", "to_node", "sign",create_using=nx.DiGraph)
    else:
        graph = nx.from_pandas_edgelist(create_df_wikipedia(), "from_node", "to_node", "sign" ,create_using=nx.DiGraph)
    return graph

In [4]:
wiki_graph = load_data_create_graph("wikiElec.ElecBs3.txt")

In [19]:
a = set((1,2))

In [22]:
a.remove(3)

In [25]:
len(a)

2

In [192]:
def compute_BwC(G,node,Community,k_node):
    sum_w = 0
    for w in Community:
        if G.has_edge(node,w):
            sum_w+=G[node][w]['sign']
        elif G.has_edge(w,node):
            sum_w+=G[w][node]['sign']
        else:
            continue
    if k_node!=0 :return sum_w/k_node
    else: return sum_w/1
    

In [193]:
from networkx.algorithms.cuts import cut_size


In [194]:
#
def compute_sigma(G,Community,weight_sum_community):
    cut = cut_size(G,Community,set(G.nodes())-Community,'sign')
    return cut/weight_sum_community

In [195]:
def compute_sum_weights(G,Community,weight_c,selected_node):
    summed = weight_c
    for node in Community:
        if G.has_edge(selected_node,node):
            summed+=G[selected_node][node]['sign']
        if G.has_edge(node,selected_node):
            summed+=G[node][selected_node]['sign']
    return summed

In [209]:
def detection_algorithm(G):
    E = set(G.edges())
    C_set = set()
    
    pos_edges = [e for e in E if G[e[0]][e[1]]['sign'] == 1]
    neg_edges = [e for e in E if G[e[0]][e[1]]['sign'] == -1]
    weight_c = 0
    while len(E)!= 0:
        print(80*'=')
        print('len(E)=',len(E))
        print(80*'=')
        if (len(pos_edges)!=0) : e=pos_edges.pop(-1)
        else: e=neg_edges.pop(-1)
        C = set()
        C.add(e[0])
        C.add(e[1])
        weight_c+=G[e[0]][e[1]]['sign']
        Nc=set()
        for v in C:
            Nc=Nc | set(G.neighbors(v))
        print(len(Nc))
        while len(Nc)!=0:
            if len(Nc)%10 == 0 : print('len(Nc) =',len(Nc))
            nodes=[]
            values=[]
            for n in C:
                k_node=0
                neighbors = [node for node in G if G.has_edge(node,n) and node not in C]
                for elem in neighbors:
                    if G.has_edge(elem,n):
                        k_node+=G[elem][n]['sign']
                    else: #G.has_edge(n,elem): 
                        k_node+=G[n][elem]['sign']
                    nodes.append(elem)
                    values.append(compute_BwC(G,elem,C,k_node))
            selected_node = nodes[np.argmax(values)]
            Nc=Nc- {selected_node}
            C1 = C | {selected_node}
            sigma_C1 = compute_sigma(G,C1,compute_sum_weights(G,C1,weight_c,selected_node))
            sigma_C = compute_sigma(G,C,weight_c)
            if sigma_C1<sigma_C :
                C=C1
            else: 
                break
        E = E - set(G.subgraph(C).edges())
        C_set = C_set | C
    return C_set

In [210]:
communities_set_wiki = detection_algorithm(wiki_graph)

len(E)= 103747
439
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 430
len(Nc) = 420
len(Nc) = 420
len(Nc) = 410
len(Nc) = 410
len(Nc) = 410
len(Nc) = 410
len(Nc) = 410
len(Nc) = 410
len(Nc) = 410
len(Nc) = 410
len(Nc) = 400
len(Nc) = 390
len(Nc) = 390
len(Nc) = 390
len(Nc) = 380
len(Nc) = 380
len(Nc) = 380
len(Nc) = 380
len(Nc) = 370
len(Nc) = 360
len(Nc) = 360
len(Nc) = 350
len(Nc) = 350
len(Nc) = 340
len(Nc) = 330
len(Nc) = 320
len(Nc) = 310
len(Nc) = 310
len(Nc) = 310
len(Nc) = 310
len(Nc) = 300
len(Nc) = 300
len(Nc) = 300
len(Nc) = 290
len(Nc) = 290
len(Nc) = 290
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 280
len(Nc) = 270
len(Nc) = 270
len(Nc) = 270
len(Nc) = 270
len(Nc) = 260
len(Nc) = 260
len(Nc) = 250
len(Nc) = 250
len(Nc) = 250
len(Nc) = 250
len(Nc) = 250
len(Nc) = 240
len(Nc) = 240
len(Nc) = 230
l

KeyboardInterrupt: 

In [ ]:
print(list(set(wiki_graph.edges()))[:10])

In [47]:
[e for e in list(set(wiki_graph.edges()))[:10] if wiki_graph[e[0]][e[1]] == 1]

[]

In [105]:
wiki_graph.has_edge(2328,3073)

False